In [1]:
import numpy as np
import Robot
from MyRobotMath import SE3

In [2]:
scara = Robot.SCARA(2,1,1)
print(scara.S_tw)
print(type(scara.S_tw))

M = scara.zero
S = scara.S_tw
B = scara.B_tw

[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, -1.0, 0.0], [0.0, 0.0, 1.0, 0.0, -2.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]
<class 'list'>


In [3]:
se3 = SE3()
w = se3.skew(scara.B_tw[0][:3])
v = scara.B_tw[0][3:]
print(w)
print(v)

[[ 0 -1  0]
 [ 1  0  0]
 [ 0  0  0]]
[0, 2, 0]


In [4]:
es_1 = se3.matexp('R',45,S[0])
es_2 = se3.matexp('R',45,S[1])
es_3 = se3.matexp('R',45,S[2])
es_4 = se3.matexp('P',0.9,S[3])
print(es_1)
matexps_s = [es_1,es_2,es_3,es_4]


[[ 0.70710678 -0.70710678  0.          0.        ]
 [ 0.70710678  0.70710678  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [5]:
eb_1 = se3.matexp('R',45,B[0])
eb_2 = se3.matexp('R',45,B[1])
eb_3 = se3.matexp('R',45,B[2])
eb_4 = se3.matexp('P',0.9,B[3])
print(eb_1)
matexps_b = [eb_1,eb_2,eb_3,eb_4]


[[ 0.70710678 -0.70710678  0.         -0.58578644]
 [ 0.70710678  0.70710678  0.          1.41421356]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [6]:
js = se3.space_jacobian(matexps_s,scara.S_tw)
print(js)

[[ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 1.          1.          1.          0.        ]
 [ 0.          0.70710678  1.70710678  0.        ]
 [ 0.         -0.70710678 -0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]


In [7]:
jb = se3.body_jacobian(M,matexps_b,matexps_s,scara.S_tw)
print(jb)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 1.70710678e+00  7.07106781e-01  3.31400081e-17  0.00000000e+00]
 [ 7.07106781e-01  7.07106781e-01 -1.67155782e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [8]:
print(se3.matsb(M,matexps_b))

[[-0.70710678 -0.70710678  0.          0.70710678]
 [ 0.70710678 -0.70710678  0.          1.70710678]
 [ 0.          0.          1.          0.9       ]
 [ 0.          0.          0.          1.        ]]


In [9]:
T_d = se3.pose_to_SE3(1,1.4,1,0,0,30)
print(T_d)

[[ 0.8660254 -0.5        0.         1.       ]
 [ 0.5        0.8660254  0.         1.4      ]
 [ 0.         0.         1.         1.       ]
 [ 0.         0.         0.         1.       ]]


In [ ]:
init = [45,45,45,0.9]

eb_1 = se3.matexp('R',init[0],B[0])
eb_2 = se3.matexp('R',init[1],B[1])
eb_3 = se3.matexp('R',init[2],B[2])
eb_4 = se3.matexp('P',init[3],B[3])

matexps_b = [eb_1,eb_2,eb_3,eb_4]

es_1 = se3.matexp('R',init[0],S[0])
es_2 = se3.matexp('R',init[1],S[1])
es_3 = se3.matexp('R',init[2],S[2])
es_4 = se3.matexp('P',init[3],S[3])

matexps_s = [es_1,es_2,es_3,es_4]

T_sb = se3.matsb(M,matexps_b)

T_bd = np.dot(np.linalg.inv(T_sb),T_d)

J_b = se3.body_jacobian(scara.zero,matexps_b,matexps_b,scara.S_tw)
J_pseudo = se3.j_inv(J_b)

V_bd = se3.relativetwist(T_bd)
print(V_bd.reshape(6,1))
theta = np.zeros(4)
theta[:3], theta[3] = np.array(np.deg2rad(init[:3])),init[3]
thetak = theta.reshape(4,1) + J_pseudo @ V_bd.reshape(6,1)
thetak[:3] = np.rad2deg(thetak[:3])
init = thetak.tolist()





[[ 0.        ]
 [ 0.        ]
 [-1.4858945 ]
 [-0.35041139]
 [-0.30708172]
 [ 0.1       ]]
[[41.47865536629368], [-70.93919138380349], [79.32505208477619], [1.0]]
